In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [51]:
m_svd = pd.read_csv("data/m_svd.csv")


In [42]:
X_train.shape + X_test.shape

(404290, 642, 2345796, 0)

In [52]:
m_svd = m_svd[:404290]

In [47]:
len(m_svd)

2345796

In [41]:
len(X_test)

2345796

In [42]:
404290+2345796

2750086

In [38]:
X_train = pd.read_csv("data/train.csv")
#X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [39]:
X_train = pd.read_csv("data/train_inter.csv")

X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())



In [6]:
X_test = pd.read_csv("data/test_inter.csv")

X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [40]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
#Y = map(lambda x: ' '.join(x), X_test['question1'])
#Y1 = map(lambda x: ' '.join(x), X_test['question2'])
hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [41]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
#Y = pd.DataFrame(hw.transform(Y).todense())
#Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [42]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
#Y = pd.DataFrame(tfidf.transform(Y).todense())
#Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [43]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
#X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
#del Y
#X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
#del Y1

In [40]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [41]:
def to_vec(x):
    result = word2vec.word_vec('yes') - word2vec.word_vec('yes')
    co = 0
    for i in x:
        if i in word2vec.vocab:
            result += word2vec.word_vec(i)
            co += 1
    return result / co

In [42]:
tmp1 = X_train['question1'].apply(lambda x: to_vec(x))
tmp2 = X_train['question2'].apply(lambda x: to_vec(x))
tmp1 = pd.DataFrame(list(tmp1))
tmp2 = pd.DataFrame(list(tmp2))
#X_train = pd.concat([X_train, pd.DataFrame(X_train['question1'].apply(lambda x: to_vec(x))),
#                  pd.DataFrame(X_train['question2'].apply(lambda x: to_vec(x)))], axis=1)
from numpy import linalg as LA

In [36]:
X_train = pd.concat([X_train, tmp1],axis=1)
X_train = pd.concat([X_train, tmp2],axis=1)

In [67]:
tmp1.fillna(tmp1.mean(), inplace=True)
tmp2.fillna(tmp2.mean(), inplace=True)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.024850,0.031879,0.073193,-0.002771,-0.066658,0.025415,0.067017,-0.147021,0.063318,0.071692,...,-0.166748,0.059119,-0.080499,-0.015268,-0.058472,-0.016852,0.026965,0.056458,0.058258,-0.045020
1,-0.024072,0.064569,0.062695,0.106104,-0.114270,-0.118555,0.030609,-0.117419,0.055848,0.058222,...,-0.068018,0.034821,-0.050604,0.013800,0.014710,0.007458,-0.011377,-0.032227,0.067801,-0.094250
2,0.002930,0.031562,0.019423,0.058431,-0.093106,0.049310,-0.008587,-0.048774,0.134705,0.088515,...,-0.047824,-0.001085,-0.058729,-0.018148,-0.075955,0.010946,0.032742,-0.046439,-0.063978,-0.073120
3,0.049748,-0.016096,0.065186,0.068412,-0.077175,-0.006226,-0.011056,-0.114886,0.079241,0.047015,...,-0.106174,0.018485,-0.029436,0.009312,-0.021572,0.003653,-0.013916,-0.005668,0.096889,-0.059187
4,-0.016357,0.122925,0.042521,0.083293,-0.067805,0.082316,-0.061096,-0.055481,0.026550,0.139038,...,-0.220947,-0.112010,-0.099772,0.016113,0.045451,0.021525,-0.011637,0.027649,0.084457,-0.060140
5,0.075684,0.019579,0.077582,0.112359,-0.105387,-0.024712,0.089444,-0.196560,0.087402,0.095608,...,0.008504,-0.014008,-0.089986,0.038093,-0.052639,-0.078274,0.054843,-0.031545,0.070608,-0.046495
6,0.028419,-0.026217,-0.006042,0.048920,-0.080688,0.035408,0.025574,-0.094559,0.092773,0.002335,...,-0.124222,0.046646,-0.090348,-0.048462,-0.056473,0.019470,0.036530,-0.010239,0.044724,-0.023849
7,-0.025675,0.024750,0.112264,0.183024,-0.074870,0.028035,0.094035,-0.055094,0.041718,0.058207,...,-0.062124,0.060221,-0.077799,0.023804,-0.043376,0.003011,0.015279,-0.095154,0.001750,-0.109121
8,0.161621,-0.017310,0.104663,0.198584,-0.100879,0.042090,0.060596,-0.092090,0.001587,0.061163,...,-0.005419,0.102051,-0.098096,-0.005176,-0.070435,-0.015283,0.068750,-0.099396,-0.039551,0.025635
9,0.074293,-0.001517,-0.025356,0.146763,-0.096784,0.124093,0.023669,-0.011034,0.107213,0.072178,...,-0.014627,0.072588,-0.053641,-0.062378,-0.053022,-0.057567,-0.023921,-0.058472,-0.094100,-0.002816


In [72]:
from numpy import linalg as LA
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis


def norm1(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [LA.norm(x.ix[i] - y.ix[i])]
    return arr


def norm2(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [cosine(x.ix[i], y.ix[i])]
    return arr


def norm3(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [jaccard(x.ix[i], y.ix[i])]
    return arr

def norm4(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [cityblock(x.ix[i], y.ix[i])]
    return arr

def norm5(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [euclidean(x.ix[i], y.ix[i])]
    return arr

def norm6(x, y):
    arr = []
    for i in x.index:
        if i % 50000 == 0:
            print(i)
        arr += [braycurtis(x.ix[i], y.ix[i])]
    return arr

In [73]:
n6 = norm6(tmp1, tmp2)

0
50000
100000
150000
200000
250000
300000
350000
400000


In [74]:
n6 = pd.DataFrame(n6)
n6.columns = ['n6']
n6.to_csv("data/n6_train.csv", index=None)

In [101]:
q_train = pd.read_csv("data/n1_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

q_train = pd.read_csv("data/n2_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

q_train = pd.read_csv("data/n3_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

q_train = pd.read_csv("data/n4_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

q_train = pd.read_csv("data/n5_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

q_train = pd.read_csv("data/n6_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [19]:
X_train[:2]

,id,qid1,qid2,question1,question2,word_match,tfidf_word_match,0,1,2,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0.727273,0.772164,0.0,0.0,0.0,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,4,"[what, is, the, story, of, kohinoor, (koh-i-no...","[what, would, happen, if, the, indian, governm...",0.307692,0.361758,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
cores_dict = pd.read_csv("data/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]
def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

#def gen_max_kcore(row):
#    return max(row["qid1_max_kcore"], row["qid2_max_kcore"])

X_train["qid1_max_kcore"] = X_train.apply(gen_qid1_max_kcore, axis=1)
#X_test["qid1_max_kcore"] = X_test.apply(gen_qid1_max_kcore, axis=1)
X_train["qid2_max_kcore"] = X_train.apply(gen_qid2_max_kcore, axis=1)
#X_test["qid2_max_kcore"] = X_test.apply(gen_qid2_max_kcore, axis=1)

In [11]:
q_test = pd.read_csv("data/kcore_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [27]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [46]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

#del X_test['question1']
#del X_test['question2']
#del X_test['test_id']

KeyError: 'question1'

In [47]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,lstm
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,-0.03559,0.150354,-0.032531,-0.086795,0.057361,0.111245,-0.033171,0.066494,0.010014,0.057908
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,-0.03081,0.003312,-0.046230,-0.044494,-0.004791,0.072093,-0.169035,-0.086499,0.006689,0.421167


In [28]:
X_train.ix[:2, 202:402]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.119495,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.131759,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.167796,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train.ix[:2, 2:202]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.113341,...,0.0,0.0,0.308628,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.165352,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.209907,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.380265,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.186087,0.0,0.0,0.0


In [30]:
#tmp = abs(X_train.ix[:, 2:202] * X_train.ix[:, -200:])
#X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])
tmp = abs(X_train.ix[:, 2:202] - X_train.ix[:, 202:402])
X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])

#tmp = abs(X_test.ix[:, 2:202] * X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])
#tmp = abs(X_test.ix[:, 2:202] - X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [77]:
tmp = abs(X_test.ix[:, 2:202] - X_test.ix[:, 202:402])
X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [85]:
del X_train

In [48]:
#X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)

In [49]:
q_train = pd.read_csv("data/f1_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [60]:
q_train = pd.read_csv("data/f2_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [50]:
q_train = pd.read_csv("data/train_lda_feat.csv", encoding = "ISO-8859-1")
q_train = q_train.ix[:, 6:]
X_train = pd.concat([X_train, q_train], axis=1)

In [53]:
X_train = pd.concat([X_train, m_svd], axis=1)

In [54]:
q_train = pd.read_csv("stacking/nn_21.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [55]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,lstm
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.105704,0.078461,-0.064659,-0.015350,0.129838,0.072293,-0.010824,0.005664,0.038002,0.588275
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.041619,-0.026479,0.073949,0.089666,0.025895,0.008982,0.057382,0.008907,-0.046267,0.203615


In [45]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041053,0.105704,0.078461,-0.064659,-0.015350,0.129838,0.072293,-0.010824,0.005664,0.038002
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.118016,-0.041619,-0.026479,0.073949,0.089666,0.025895,0.008982,0.057382,0.008907,-0.046267


In [69]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [90]:
del tmp

NameError: name 'tmp' is not defined

In [13]:
import gc
gc.collect()

111

In [14]:
q_test = pd.read_csv("data/freq_test.csv")


In [15]:
X_test = pd.concat([X_test, q_test], axis=1)

In [16]:
f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]

In [17]:
X_test = pd.concat([X_test, f_test], axis=1)

In [18]:
q_test = pd.read_csv("data/f1_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [19]:
q_test = pd.read_csv("data/f2_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [19]:
q_test = pd.read_csv("data/test_lda_feat.csv", encoding = "ISO-8859-1")
q_test = q_test.ix[:, 3:]
X_test = pd.concat([X_test, q_test], axis=1)

In [23]:
X_test.shape

(2345796, 442)

In [29]:
m_svd.index = range(len(m_svd))

In [31]:
m_svd.shape

(2345796, 30)

In [32]:
X_test = pd.concat([X_test, m_svd], axis=1)

In [33]:
q_test = pd.read_csv("stacking/nn_21_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [75]:
X_train

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,lstm
0,0.727273,0.772164,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.105704,0.078461,-0.064659,-0.015350,0.129838,0.072293,-0.010824,0.005664,0.038002,0.588275
1,0.307692,0.361758,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,-0.041619,-0.026479,0.073949,0.089666,0.025895,0.008982,0.057382,0.008907,-0.046267,0.203615
2,0.363636,0.355191,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.009619,0.076655,-0.060903,0.028583,0.001339,0.026398,-0.008559,0.008202,-0.059601,0.476624
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.219821,0.000000,0.000000,...,-0.000195,-0.060413,0.087814,0.055352,-0.030849,0.016903,-0.082594,0.047540,-0.174111,0.016573
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,-0.009776,-0.077315,-0.056978,0.069541,0.065167,0.106326,-0.027191,0.077681,0.005758,0.004713
5,0.470588,0.510771,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,-0.019854,0.110946,0.036313,0.024900,-0.008638,0.016371,-0.010809,0.034267,-0.015158,0.469059
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,-0.101719,0.049948,-0.015352,-0.030019,0.083443,0.022006,-0.028432,-0.078518,-0.038912,0.070437
7,0.500000,0.645836,0.000000,0.000000,0.000000,0.000000,0.0,-0.301252,0.000000,0.000000,...,-0.126262,0.033150,-0.020414,-0.084632,0.031626,-0.077152,0.040474,-0.009557,-0.168270,0.784010
8,0.500000,0.396755,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.059640,-0.007842,-0.174651,-0.050179,0.007329,-0.025207,-0.015739,-0.004169,-0.016590,0.304295
9,0.363636,0.503203,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,-0.045970,-0.010590,0.116779,0.055601,-0.084238,0.028893,-0.013940,0.009434,-0.183440,0.173784


In [74]:
X_test

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,lstm
1000001,0.600000,0.528512,0.000000,0.000000,0.000000,0.000000,0.000000,-0.163117,0.0,0.000000,...,-0.065232,0.042553,0.125522,-0.001698,0.049479,0.024954,-0.049810,-0.099132,0.081386,0.218268
1000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.083686,0.037723,-0.006557,0.010708,0.044189,-0.055917,0.117250,-0.107494,0.077014,0.035791
1000003,0.222222,0.236190,0.000000,0.000000,0.000000,0.000000,0.000000,-0.237763,0.0,0.000000,...,-0.065046,0.048662,-0.066979,-0.043595,0.001533,-0.142618,-0.008402,0.067051,-0.001722,0.008564
1000004,0.333333,0.325022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.148396,0.019530,0.085037,0.141574,-0.058321,-0.087291,0.019283,-0.019269,-0.058823,0.093719
1000005,0.125000,0.091719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.326874,...,0.032391,-0.021011,-0.019613,0.034625,0.000527,0.030112,-0.122613,0.004838,0.011651,0.023209
1000006,0.363636,0.389526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.063871,0.001868,0.168888,0.055078,0.051125,-0.013163,0.013484,-0.062062,0.011989,0.569536
1000007,0.086957,0.092300,-0.261481,0.000000,-0.448088,0.000000,0.000000,0.116051,0.0,0.000000,...,0.090015,0.158018,-0.007469,0.049505,-0.044311,0.026383,-0.004805,0.021606,0.003633,0.037642
1000008,0.545455,0.510289,0.000000,0.000000,0.000000,0.000000,0.000000,-0.249636,0.0,0.000000,...,0.014706,-0.044525,-0.129258,-0.088549,0.014168,0.002851,-0.084417,-0.035991,0.108509,0.046387
1000009,0.400000,0.238264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.093083,0.036975,0.091758,0.084157,0.079570,0.001644,0.028117,-0.083923,0.028721,0.113509
1000010,0.210526,0.209631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.089231,-0.080225,-0.076387,0.012923,0.098360,0.025772,0.011429,-0.012146,0.007215,0.089301


In [35]:
len(X_test)

2345796

In [83]:
len(m_svd)

2345796

In [19]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=1500, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred

    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.219 +- 0.002


In [7]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [21]:
pred_train = foo(pred_train)

In [22]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_22_1.csv", index=None)

In [58]:
del pred_train

In [23]:
import gc
gc.collect()

255

In [10]:
submit = pd.read_csv("data/sample_submission.csv")

In [56]:
#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

In [58]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=1500, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=100,
        min_split_gain=0, n_estimators=1500, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [61]:
X_test = X_test.ix[1000001:, :]

In [62]:
test_pred2 = lgb.predict_proba(X_test)[:, 1]

In [60]:
test_pred1[:4]

array([ 0.00115463,  0.20790386,  0.34077708,  0.00080505])

In [127]:
test_pred1[-4:]

array([  4.85291907e-04,   2.61291438e-01,   6.26312391e-01,
         1.66797416e-03])

In [63]:
test_pred2[:4]

array([  1.49419076e-01,   2.03473097e-05,   5.70862511e-05,
         9.43047064e-02])

In [151]:
test_pred2[-4:]

array([  1.67587699e-03,   2.31711933e-04,   3.72899384e-01,
         3.89707027e-01])

In [148]:
len(test_pred1)

1000001

In [152]:
len(test_pred2)

1345795

In [64]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [65]:
len(test_pred)

2345796

In [66]:
test_pred

array([  1.15462523e-03,   2.07903865e-01,   3.40777080e-01, ...,
         9.96986715e-05,   2.12297124e-01,   5.79660122e-01])

In [146]:
del pred_train

NameError: name 'pred_train' is not defined

In [63]:
import gc
gc.collect()

0

In [38]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

In [39]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [40]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [62]:
del Xdatatest2

In [57]:
Xdatatest2 = xgb.DMatrix(data=X_test.ix[1000001:, :].values)

In [42]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [45]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.633506	eval-logloss:0.646774
[15]	train-logloss:0.323969	eval-logloss:0.388133
[30]	train-logloss:0.271702	eval-logloss:0.343325
[45]	train-logloss:0.257076	eval-logloss:0.341127
[60]	train-logloss:0.248104	eval-logloss:0.316478
[75]	train-logloss:0.241793	eval-logloss:0.306781
[90]	train-logloss:0.236545	eval-logloss:0.294224
[105]	train-logloss:0.232073	eval-logloss:0.288613
[120]	train-logloss:0.228825	eval-logloss:0.282442
[135]	train-logloss:0.225883	eval-logloss:0.283732
[150]	train-logloss:0.222672	eval-logloss:0.279839
[165]	train-logloss:0.220304	eval-logloss:0.271811
[180]	train-logloss:0.218097	eval-logloss:0.271271
[195]	train-logloss:0.215673	eval-logloss:0.269905
[210]	train-logloss:0.213894	eval-logloss:0.270819
[225]	train-logloss:0.211979	eval-logloss:0.273126
[240]	train-logloss:0.210103	eval-logloss:0.267782
[255]	train-logloss:0.208528	eval-logloss:0.267423
[270]	train-logloss:0.207038	eval-logloss:0.269102
[285]	train-logloss:0.205572	eval-loglo

In [61]:
test_pred2 = bst.predict(Xdatatest2)


In [54]:
test_pred1[:4]

array([  4.93649067e-03,   1.69867009e-01,   5.44634938e-01,
         2.05250995e-04], dtype=float32)

In [50]:
len(test_pred1)

1000001

In [53]:
test_pred1[:4]

array([  2.29527755e-03,   5.45756638e-01,   6.37812853e-01,
         4.23383579e-04], dtype=float32)

In [64]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [65]:
len(test_pred)

2345796

In [2]:
test_pred = pd.read_csv("stacking/nn_21_test.csv")


In [5]:
test_pred = test_pred['lstm']

In [8]:
test_pred = foo(test_pred)

In [11]:
submit['is_duplicate'] = test_pred

In [12]:
submit.to_csv("stacking/nn_tmp.csv", index=None)

In [13]:
submit

,test_id,is_duplicate
0,0,0.020263
1,1,0.196667
2,2,0.327514
3,3,0.001061
4,4,0.464111
5,5,0.009520
6,6,0.393998
7,7,0.838837
8,8,0.326061
9,9,0.004459


In [161]:
submit

,test_id,is_duplicate
0,0,0.002633
1,1,0.323567
2,2,0.218058
3,3,0.000561
4,4,0.095342
5,5,0.007831
6,6,0.956379
7,7,0.098221
8,8,0.363882
9,9,0.002275
